# Similarities & Distances

0. Поиск ключевых слов по 4-м тематикам
- спорт
- наука
- шоппинг
- новости

На просторах куча сайтов с базами данных ключевых слов, но, of course, почти все они платные.... :sob:

Сначала я хотела составить базу слов, основываясь на данных с сервисов Google Trends (у этого сервиса удобнее интерфейс и есть возможность скачать csv с related запросами) и Yandex Wordstat. Но они предоставляют ограниченное кол-во слов, и в большинстве случаев предоставляемые слова (по моему субъективному мнению) не очень хорошо описывают данный топик.

Поэтому я использовала сервис https://kartaslov.ru/, на этом сайте можно получить синонимичные слова, слова, которые ассоциируются с данным топиком - что является более характеризующим/описательным для определения принадлежности какого-то текста к данному топику.

Слова находятся в директории data/topics.


1. Prepocessing
Токенизация, лемматизация и удаление стоп-слов и символов пунктуации

In [21]:
import nltk
from nltk.tokenize import word_tokenize

In [22]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/renett/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/renett/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Токенизируем и убираем стоп-слова и пунктуацию

In [31]:
file_name = "data/oreologia.txt"

with open(file_name, 'r') as reader:
    text = reader.readlines()


tokens = []
for line in text:
    tokens += word_tokenize(line)

print(tokens)
print(len(tokens))

['Ореология', ':', 'механика', 'печенья', 'и', 'крема', 'Oreo', 'Представители', 'современного', 'научного', 'сообщества', 'пытаются', 'найти', 'решения', 'многих', 'проблем', ',', 'что', 'разительно', 'улучшило', 'бы', 'жизнь', 'человека', '.', 'Поиски', 'новых', 'методов', 'лечения', 'рака', ',', 'оценка', 'эффективности', 'новых', 'источников', 'энергии', ',', 'открытие', 'новых', 'планет', '—', 'все', 'это', 'крайне', 'важно', 'и', 'невероятно', 'сложно', '.', 'Тем', 'не', 'менее', 'иногда', 'ученые', 'задаются', 'вопросами', ',', 'которые', 'звучат', 'так', ',', 'словно', 'возникли', 'в', 'голове', 'любопытного', 'ребенка', 'дошкольного', 'возраста', '.', 'Исследователи', 'из', 'Массачусетского', 'технологического', 'института', '(', 'США', ')', 'задали', 'такой', '—', 'почему', 'при', 'разъединении', 'половинок', 'печенья', 'Oreo', 'крем', 'в', '95', '%', 'случаев', 'полностью', 'остается', 'только', 'на', 'одной', 'половинке', '?', 'Чтобы', 'ответить', 'на', 'этот', 'вопрос', ',

In [58]:
# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords
from string import punctuation

noise = stopwords.words('russian') + list(punctuation)
noise.append("т.")
noise.append("д")
noise.append("—")
noise.append("«")
noise.append("»")

print(noise)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [61]:
filtered_tokens = [x for x in tokens if x.lower() not in noise]

print(len(filtered_tokens))
print(filtered_tokens)
# 2186 tokens left

2186
['Ореология', 'механика', 'печенья', 'крема', 'Oreo', 'Представители', 'современного', 'научного', 'сообщества', 'пытаются', 'найти', 'решения', 'многих', 'проблем', 'разительно', 'улучшило', 'жизнь', 'человека', 'Поиски', 'новых', 'методов', 'лечения', 'рака', 'оценка', 'эффективности', 'новых', 'источников', 'энергии', 'открытие', 'новых', 'планет', 'это', 'крайне', 'важно', 'невероятно', 'сложно', 'менее', 'ученые', 'задаются', 'вопросами', 'которые', 'звучат', 'словно', 'возникли', 'голове', 'любопытного', 'ребенка', 'дошкольного', 'возраста', 'Исследователи', 'Массачусетского', 'технологического', 'института', 'США', 'задали', 'почему', 'разъединении', 'половинок', 'печенья', 'Oreo', 'крем', '95', 'случаев', 'полностью', 'остается', 'одной', 'половинке', 'ответить', 'вопрос', 'потребовалось', 'провести', 'довольно', 'серьезные', 'расчеты', 'эксперименты', 'Свой', 'труд', 'ученые', 'назвали', 'говорящим', 'именем', 'ореология', 'показали', 'расчеты', 'опыты', 'какие', 'физичес

Лемматизируем (получаем начальные формы слов)

In [62]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

lemmas = []
for token in filtered_tokens:
    lem = pymorphy2_analyzer.parse(token)
    lemmas.append(lem[0].normal_form)
print(lemmas)

['ореология', 'механика', 'печение', 'крем', 'oreo', 'представитель', 'современный', 'научный', 'сообщество', 'пытаться', 'найти', 'решение', 'многий', 'проблема', 'разительно', 'улучшить', 'жизнь', 'человек', 'поиск', 'новый', 'метод', 'лечение', 'рак', 'оценка', 'эффективность', 'новый', 'источник', 'энергия', 'открытие', 'новый', 'планета', 'это', 'крайне', 'важно', 'невероятно', 'сложно', 'менее', 'учёный', 'задаваться', 'вопрос', 'который', 'звучать', 'словно', 'возникнуть', 'голова', 'любопытный', 'ребёнок', 'дошкольный', 'возраст', 'исследователь', 'массачусетский', 'технологический', 'институт', 'сша', 'задать', 'почему', 'разъединение', 'половинка', 'печение', 'oreo', 'крем', '95', 'случай', 'полностью', 'оставаться', 'один', 'половинка', 'ответить', 'вопрос', 'потребоваться', 'провести', 'довольно', 'серьёзный', 'расчёт', 'эксперимент', 'свой', 'труд', 'учёный', 'назвать', 'говорящий', 'имя', 'ореология', 'показать', 'расчёт', 'опыт', 'какой', 'физический', 'сила', 'воздейств

2.

IndexError: list index out of range